In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/11 06:09:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_green=spark.read.parquet('../data/part/green/*/*')

In [4]:
df_green.createOrReplaceTempView('green')

In [5]:
df_green_revenue = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS zone,
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
GROUP BY
    1, 2
""")


In [6]:
df_green_revenue.show()

+----+-------------------+------------------+--------------+
|zone|               hour|            amount|number_records|
+----+-------------------+------------------+--------------+
|   7|2020-01-01 17:00:00|357.84000000000015|            28|
| 127|2020-01-02 06:00:00|             92.16|             2|
|  65|2020-01-21 23:00:00|            147.64|            10|
| 228|2020-01-09 12:00:00|235.15000000000003|            11|
|  66|2020-01-26 21:00:00|             80.83|             5|
|  75|2020-01-23 23:00:00|394.58000000000015|            25|
|  42|2020-01-07 23:00:00|            147.55|            10|
|  83|2020-01-19 00:00:00|             76.91|             6|
| 130|2020-01-17 13:00:00|200.17000000000002|            10|
|  95|2020-01-17 07:00:00|            246.16|            14|
| 152|2020-01-12 09:00:00|            136.09|             6|
|  95|2020-01-28 12:00:00|428.55000000000007|            25|
| 210|2020-01-10 21:00:00|            156.28|             6|
|  74|2020-01-30 17:00:0

In [7]:
df_green_revenue.write.parquet('../data/report/revenue/green', mode='overwrite')

In [8]:
df_yellow=spark.read.parquet('../data/part/yellow/*/*')

In [9]:
df_yellow.createOrReplaceTempView('yellow')

In [10]:
df_yellow_revenue = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS zone,
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
GROUP BY
    1, 2
""")

In [11]:
df_yellow_revenue.write.parquet('../data/report/revenue/yellow', mode='overwrite')

In [15]:
df_green_revenue_tmp=df_green_revenue.withColumnRenamed('amount', 'green_amount').withColumnRenamed('number_records','green_number_records')

df_yellow_revenue_tmp=df_yellow_revenue.withColumnRenamed('amount', 'yellow_amount').withColumnRenamed('number_records','yellow_number_records')

In [20]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour','zone'], how='outer')

In [22]:
df_join.write.parquet('../data/report/revenue/total', mode='overwrite')

In [23]:
df_join.show()

+-------------------+----+------------+--------------------+------------------+---------------------+
|               hour|zone|green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------+--------------------+------------------+---------------------+
|2020-01-01 04:00:00|   1|        null|                null|              94.8|                    1|
|2020-01-01 08:00:00|   1|        null|                null|              60.8|                    1|
|2020-01-01 11:00:00|   1|        null|                null|              90.3|                    1|
|2020-01-01 12:00:00|   1|        null|                null|            266.66|                    3|
|2020-01-01 13:00:00|   1|        null|                null|            213.36|                    2|
|2020-01-01 20:00:00|   1|        null|                null|             96.96|                    3|
|2020-01-02 15:00:00|   1|        null|                null|246.60000000000002|   

In [24]:
df_join = spark.read.parquet('../data/report/revenue/total')

In [25]:
df_zones = spark.read.parquet('zones')

In [26]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [28]:
df_result=df_join.join(df_zones,df_join.zone == df_zones.LocationID)

In [30]:
df_result.drop('LocationID','zone').write.parquet('tmp/revenue-zones')